# Creation des cartes folium

In [ ]:
import pandas as pd
haies = pd.read_csv('1_haies.csv', delimiter=',',index_col=0)
canaux = pd.read_csv('1_canaux.csv', delimiter=',',index_col=0)
routes = pd.read_csv('1_routes.csv', delimiter=',',index_col=0)

In [ ]:
import folium
import pandas as pd
import pyproj
import json
from branca.element import Template, MacroElement
import geopandas as gpd

# Convertir les coordonnées Lambert II étendu en coordonnées latitude/longitude
lambert = pyproj.Proj("+init=EPSG:27572")
wgs84 = pyproj.Proj("+init=EPSG:4326")

# Convertir les coordonnées des haies en latitude/longitude
A_haies = haies[['X1', 'Y1']].values
B_haies = haies[['X2', 'Y2']].values
# Convertir les coordonnées des canaux en latitude/longitude
A_canaux = canaux[['X1', 'Y1']].values
B_canaux = canaux[['X2', 'Y2']].values
# Convertir les coordonnées des routes en latitude/longitude
A_routes = routes[['X1', 'Y1']].values
B_routes = routes[['X2', 'Y2']].values

#On transforme les coordonnées des haies en wgs84
A_haies_latlon = pyproj.transform(lambert, wgs84, A_haies[:, 0], A_haies[:, 1])
B_haies_latlon = pyproj.transform(lambert, wgs84, B_haies[:, 0], B_haies[:, 1])
#On transforme les coordonnées des canaux en wgs84
A_canaux_latlon = pyproj.transform(lambert, wgs84, A_canaux[:, 0], A_canaux[:, 1])
B_canaux_latlon = pyproj.transform(lambert, wgs84, B_canaux[:, 0], B_canaux[:, 1])
#On transforme les coordonnées des routes en wgs84
A_routes_latlon = pyproj.transform(lambert, wgs84, A_routes[:, 0], A_routes[:, 1])
B_routes_latlon = pyproj.transform(lambert, wgs84, B_routes[:, 0], B_routes[:, 1])

# Créer une carte centrée sur la Bretagne
m = folium.Map(location=[48.495722, -1.578333], zoom_start=12)


# Créer une couche pour les haies
haies_folium = folium.FeatureGroup(name='haies_folium')
m.add_child(haies_folium)
# Ajouter les segments des haies à la couche haies_folium avec les extrémités
for i in range(len(haies)):
    start = [A_haies_latlon[1][i], A_haies_latlon[0][i]]
    end = [B_haies_latlon[1][i], B_haies_latlon[0][i]]
    folium.PolyLine(locations=[start, end], color='green').add_to(haies_folium)
    # Ajouter un marqueur pour l'extrémité de départ
    folium.CircleMarker(location=start, radius=5, color='green', fill=True, fill_color='green', fill_opacity=0.5).add_to(haies_folium)
    folium.CircleMarker(location=end, radius=5, color='green', fill=True, fill_color='green', fill_opacity=0.5).add_to(haies_folium)

# Créer une couche pour les canaux
canaux_folium = folium.FeatureGroup(name='canaux_folium')
m.add_child(canaux_folium)
# Ajouter les segments des canaux à la couche canaux_folium avec les extrémités
for i in range(len(canaux)):
    start = [A_canaux_latlon[1][i], A_canaux_latlon[0][i]]
    end = [B_canaux_latlon[1][i], B_canaux_latlon[0][i]]
    folium.PolyLine(locations=[start, end], color='blue').add_to(canaux_folium)
    # Ajouter un marqueur pour l'extrémité de départ
    folium.CircleMarker(location=start, radius=5, color='blue', fill=True, fill_color='blue', fill_opacity=0.5).add_to(canaux_folium)
    folium.CircleMarker(location=end, radius=5, color='blue', fill=True, fill_color='blue', fill_opacity=0.5).add_to(canaux_folium)

# Créer une couche pour les routes
routes_folium = folium.FeatureGroup(name='routes_folium')
m.add_child(routes_folium)
# Ajouter les segments des routes à la couche routes_folium avec les extrémités
for i in range(len(routes)):
    start = [A_routes_latlon[1][i], A_routes_latlon[0][i]]
    end = [B_routes_latlon[1][i], B_routes_latlon[0][i]]
    folium.PolyLine(locations=[start, end], color='red').add_to(routes_folium)
    # Ajouter un marqueur pour l'extrémité de départ
    folium.CircleMarker(location=start, radius=5, color='red', fill=True, fill_color='red', fill_opacity=0.5).add_to(routes_folium)
    folium.CircleMarker(location=end, radius=5, color='red', fill=True, fill_color='red', fill_opacity=0.5).add_to(routes_folium)

# Ajouter les couches à la carte
folium.LayerControl().add_to(m)

template = """
{% macro html(this, kwargs) %}

<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <title>Carte du Projet</title>
  <link rel="stylesheet" href="//code.jquery.com/ui/1.12.1/themes/base/jquery-ui.css">
  <script src="https://code.jquery.com/jquery-1.12.4.js"></script>
  <script src="https://code.jquery.com/ui/1.12.1/jquery-ui.js"></script>
  
  <script>
  $( function() {
    $( "#maplegend" ).draggable({
                    start: function (event, ui) {
                        $(this).css({
                            right: "auto",
                            top: "auto",
                            bottom: "auto"
                        });
                    }
                });
});
  </script>
</head>

#Corps du fichier html
<body>
 
<div id='maplegend' class='maplegend' 
    style='position: absolute; z-index:9999; border:2px solid grey; background-color:rgba(255, 255, 255, 0.8);
     border-radius:6px; padding: 10px; font-size:14px; left: 80px; top: 50px;'>
<div class='legend-title'>Visualisation des Linéaires de Bretagne </div>
<div class='legend-scale'>
  <ul class='legend-labels'>
    <li><span style='background:green;opacity:0.7;'></span>Haies</li>
    <li><span style='background:red;opacity:0.7;'></span>Routes</li>
    <li><span style='background:blue;opacity:0.7;'></span>Canaux</li>
  </ul>
</div>
</div>
 
</body>
</html>
<style type='text/css'>
  .maplegend .legend-title {
    text-align: center;
    margin-bottom: 5px;
    font-weight: bold;
    font-size: 90%;
    }
  .maplegend .legend-scale ul {
    margin: 0;
    margin-bottom: 5px;
    padding: 0;
    float: left;
    list-style: none;
    }
  .maplegend .legend-scale ul li {
    font-size: 80%;
    list-style: none;
    margin-left: 0;
    line-height: 18px;
    margin-bottom: 2px;
    }
  .maplegend ul.legend-labels li span {
    display: block;
    float: left;
    height: 16px;
    width: 30px;
    margin-right: 5px;
    margin-left: 0;
    border: 1px solid #999;
    }
  .maplegend .legend-source {
    font-size: 80%;
    color: #777;
    clear: both;
    }
  .maplegend a {
    color: #777;
    }
</style>
{% endmacro %}"""

macro = MacroElement()
macro._template = Template(template)
m.get_root().add_child(macro)

# Enregistrer la carte
m.save('2_carte_bretagne.html')

c:\Users\cypri\AppData\Local\Programs\Python\Python310\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
c:\Users\cypri\AppData\Local\Programs\Python\Python310\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\cypri\AppData\Local\Temp\ipykernel_18476\705338347.py:23: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotc